<img src='../images/cards.png' width='120px' align='right' style="padding: 15px">


# Protected and "Private" variables

## Goal

This notebook shall demonstrate protected and "private" variables, as well as a short introduction to decorators. 

- [Protected vs. Private variable](#protected)
- [Decorators](#decorators)
- [Disclaimer*](#disclaimer)

*Note that the use of "private" variables in Python is not strictly common practice, for reasons that will be explained in this notebook.

<a id=protected></a>

## Protected vs. Private variables

All variables prefixed with an `_` are protected. It's advised that you don't use them outside of the class, as they are prone to change upon a new version of the library etc. Private variables are prefixed with `__` and cannot be use outside of the class.

For example, let's add a double underscore before the `cards` attribute

In [ ]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class Deck:
    ranks = '23456789TJQKA'
    suits = '♠♥♦♣'
    
    def __init__(self, debug_value = False):
        self.__cards = [
            Card(rank, suit)
            for suit in self.suits
            for rank in self.ranks
        ]
   
    def __len__(self):
        return len(self._cards)
        
    def __str__(self):
        return f'Deck(suits={self.suits}, ranks={self.ranks})'
    
    def __getitem__(self, position):
        return self._cards[position]
    
    def __setitem__(self, ind, value):
        self.__cards[ind] = value
    
    def deal(self):
        return self._cards.pop()
    
    def check_ace(self):
        return self.cards[-1].rank == 'A'
        
deck = Deck()

In [ ]:
# NBVAL_RAISES_EXCEPTION
# deck.__cards

Now you can't access `__cards` outside of the class. So how would you check the list of cards? Well you could implement a method for that.

### <mark>Exercise: Implement a method to check the cards</mark>

Add a new method called `cards` that will simply return the list of cards.

**Bonus**: Add the decorator `@property` on the line above your new method. What does that mean for how you look at the cards?

In [ ]:
class Deck:
    ranks = '23456789TJQKA'
    suits = '♠♥♦♣'
    
    def __init__(self, debug_value = False):
        self.__cards = [
            Card(rank, suit)
            for suit in self.suits
            for rank in self.ranks
        ]
   
    def __len__(self):
        return len(self.__cards)
        
    def __str__(self):
        return f'Deck(suits={self.suits}, ranks={self.ranks})'
    
    def __getitem__(self, position):
        return self.__cards[position]
    
    def __setitem__(self, ind, value):
        self.__cards[ind] = value
    
    def deal(self):
        return self.__cards.pop()
    
    def check_ace(self):
        return self.__cards[-1].rank == 'A'
    
    ## Add method here
    
        
deck = Deck()

**Answers**

In [ ]:
# %load ../answers/ex-cards-method.py

<a id=decorators></a>
## Decorators

The use of the `@property` decorator allows us to treat the method as if it were an attribute. This way of working is adopted in lots of common classes in Python, most commonly used by data scientists/analysts is the [pandas.DataFrame.shape](https://github.com/pandas-dev/pandas/blob/main/pandas/core/frame.py#L833) attribute. 

Look what happens if we tried to overwrite `deck.cards`

In [ ]:
class Deck:
    ranks = '23456789TJQKA'
    suits = '♠♥♦♣'
    
    def __init__(self, debug_value = False):
        self.__cards = [
            Card(rank, suit)
            for suit in self.suits
            for rank in self.ranks
        ]
   
    def __len__(self):
        return len(self.__cards)
        
    def __str__(self):
        return f'Deck(suits={self.suits}, ranks={self.ranks})'
    
    def __getitem__(self, position):
        return self.__cards[position]
    
    def __setitem__(self, ind, value):
        self.__cards[ind] = value
    
    def deal(self):
        return self.__cards.pop()
    
    def check_ace(self):
        return self.__cards[-1].rank == 'A'
    
    @property
    def cards(self):
        return self.__cards
        
deck = Deck()

In [ ]:
# deck.cards = 'Overwrite the attribute'

Now the attribute `cards` is very safe, but the user can still access the deck to check all the cards in it. Now if the user were to create a subclass and create a new attribute called `cards` the original attribute would remain untouched. 

There are many other powerful decorators in the Python library that can be used:
- `@cache`: cache the results of your function
- `@lru_cache`: cache the result of your functions using the Least Recently Used (LRU) strategy
- `@singledispatch`: transform your regular function into a single dispatch generic function - [more info here](https://www.blog.pythonlibrary.org/2016/02/23/python-3-function-overloading-with-singledispatch)

<a id=disclaimer></a>

## <mark style='background-color:#1EB0E0;color:white'>Disclaimer: A note on "privacy" in Python</mark>


While this is an interesting look at how to avoid mangling variable in Python, it is not a common practice. Unlike Java with the private modifier, there **is no way to create private variables in Python**. What we have in Python is a simple mechanism to prevent accidental overwriting of a “private” attribute in a subclass.

Ian Bicking (creator of pip, virtualenv, and other projects) wrote:

> Never, ever use two leading underscores. This is annoyingly private. If name clashes are a concern, use explicit name mangling instead (e.g., _MyThing_blahblah). This is essentially the same thing as double-underscore, only it’s transparent where double underscore obscures.7

Therefore a better practice would be to use the single leading underscore, as it still convinces the user to not use that attribute outside the class, and is not hidden/obscured. 

Let's rename the `__cards` attribute to fit the philosophy of Ian Bicking:

In [ ]:
class Deck:
    ranks = '23456789TJQKA'
    suits = '♠♥♦♣'
    
    def __init__(self, debug_value = False):
        self._original_cards = [
            Card(rank, suit)
            for suit in self.suits
            for rank in self.ranks
        ]
   
    def __len__(self):
        return len(self._original_cards)
        
    def __str__(self):
        return f'Deck(suits={self.suits}, ranks={self.ranks})'
    
    def __getitem__(self, position):
        return self._original_cards[position]
    
    def __setitem__(self, ind, value):
        self._original_cards[ind] = value
    
    def deal(self):
        return self._original_cards.pop()
    
    def check_ace(self):
        return self._original_cards[-1].rank == 'A'
    
    @property
    def cards(self):
        return self._original_cards
        
deck = Deck()

The attribute `_original_cards` is still accessible, and not hidden/obscured:

In [ ]:
print(deck._original_cards)

But the leading underscore indicates to the user to note use this attribute, instead using the method (with the `@property` decorator) is a better way to access this attribute:

In [ ]:
print(deck.cards)

The philosophy around "private" variables has been taken from ***Fluent Python, chapter: A Pythonic Object, section: Private and “Protected” Attributes in Python***